<h1><u><b>FEATURE SELECTION</b></u></h1>

This notebook contains the feature engineering step before building the model
we did basic feature elimination in the previous notebook considering, eliminating the features which would hardly contribute towards our prediction.
We have 15 features in our data, training the model with these 15 features can overfit out model and can result in poor predictions on the test set.Therefore a more advanced technique should be emplyed to further select the appropriate features.
In this notebook we move towards more advanced feature selection techniques like the <b>Baruta feature selection algorithm</b> using random forest classifier as backend.  

In [1]:
import collections
import pandas
from collections import defaultdict
import numpy as np 
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from numpy.random import permutation
import math

In [180]:
data=pd.read_csv('processed_data.csv')

In [181]:
data

,Unnamed: 0,userName,major,researchExp,industryExp,toeflScore,internExp,greV,greQ,journalPubs,greA,termAndYear,confPubs,ugCollege,cgpa,univName
0,4,ninny.rgs,electricalengineeringandcomputerscience,0.0,0.0,101.0,0.0,430.0,740.0,0.0,4.5,Fall - 2011,0.0,abhinavhitechcollegeofengineering,0.7227,University of Southern California
1,5,ninny.rgs,electricalengineeringandcomputerscience,0.0,0.0,101.0,0.0,159.0,159.0,0.0,4.5,Fall - 2011,0.0,abviiitm,0.7227,New Jersey Institute of Technology
2,8,aashayshah,masterofengineeringmanagement,0.0,7.0,88.0,0.0,147.0,164.0,0.0,2.5,Spring - 2016,0.0,aceengineeringcollege,0.6200,University of Texas Arlington
3,13,cjaganmohan,electronicsandcommunication,0.0,0.0,102.0,0.0,140.0,148.0,0.0,3.0,Fall - 2013,0.0,acharyainstituteoftechnology,0.7400,University of Texas Arlington
4,16,pvskmz,computerscience,0.0,0.0,115.0,0.0,154.0,163.0,0.0,3.0,Fall - 2013,0.0,acharyanagarjunauniversity,0.7100,Arizona State University
5,17,anu_661,electricalengineeringandcomputerscience,0.0,0.0,105.0,0.0,154.0,163.0,0.0,3.0,Fall - 2011,0.0,acharyanagarjunauniversity,0.8900,SUNY Stony Brook
6,20,nachiappan123,electricalengineering,0.0,45.0,100.0,0.0,157.0,163.0,0.0,3.0,Fall - 2015,0.0,acharyanagarjunauniversity,0.8004,Arizona State University
7,21,username123456,managementinformationsystem,0.0,0.0,95.0,0.0,143.0,162.0,0.0,3.0,Spring - 2014,0.0,acharyanagarjunauniversity,0.8500,University of Texas Arlington
8,22,username123456,engineeringmanagement,0.0,0.0,95.0,0.0,143.0,162.0,0.0,3.0,Spring - 2014,0.0,acharyanagarjunauniversity,0.8500,University of North Carolina Charlotte
9,23,sam_the_killer,electricalandcomputerengineeringelectricalengi...,0.0,0.0,89.0,0.0,370.0,690.0,0.0,4.0,Fall - 2007,0.0,acharyanagarjunauniversity,0.6941,University of Texas Arlington


In [182]:
data=data.drop('Unnamed: 0',1)

In [183]:
data.isnull().sum()

userName          0
major          1737
researchExp       0
industryExp       0
toeflScore        0
internExp         0
greV              0
greQ              0
journalPubs       0
greA              0
termAndYear       0
confPubs          0
ugCollege         0
cgpa              0
univName          0
dtype: int64

In [184]:
#entries with no major specified can be considered outliers and should be removed.
data=data.dropna()

In [185]:
data.isnull().sum()

userName       0
major          0
researchExp    0
industryExp    0
toeflScore     0
internExp      0
greV           0
greQ           0
journalPubs    0
greA           0
termAndYear    0
confPubs       0
ugCollege      0
cgpa           0
univName       0
dtype: int64

In [186]:
#considering only the universities which have more than 100 entries
university_list = list(set(data['univName'].tolist()))
for i in range(len(university_list)):
    if(len(data[data['univName'] == university_list[i]]) < 100):
        data = data[data['univName'] != university_list[i]]

In [187]:
data['univName'].nunique()

43

In [188]:
result=data.copy()

In [189]:
data

,userName,major,researchExp,industryExp,toeflScore,internExp,greV,greQ,journalPubs,greA,termAndYear,confPubs,ugCollege,cgpa,univName
0,ninny.rgs,electricalengineeringandcomputerscience,0.0,0.0,101.0,0.0,430.0,740.0,0.0,4.5,Fall - 2011,0.0,abhinavhitechcollegeofengineering,0.7227,University of Southern California
1,ninny.rgs,electricalengineeringandcomputerscience,0.0,0.0,101.0,0.0,159.0,159.0,0.0,4.5,Fall - 2011,0.0,abviiitm,0.7227,New Jersey Institute of Technology
2,aashayshah,masterofengineeringmanagement,0.0,7.0,88.0,0.0,147.0,164.0,0.0,2.5,Spring - 2016,0.0,aceengineeringcollege,0.6200,University of Texas Arlington
3,cjaganmohan,electronicsandcommunication,0.0,0.0,102.0,0.0,140.0,148.0,0.0,3.0,Fall - 2013,0.0,acharyainstituteoftechnology,0.7400,University of Texas Arlington
4,pvskmz,computerscience,0.0,0.0,115.0,0.0,154.0,163.0,0.0,3.0,Fall - 2013,0.0,acharyanagarjunauniversity,0.7100,Arizona State University
5,anu_661,electricalengineeringandcomputerscience,0.0,0.0,105.0,0.0,154.0,163.0,0.0,3.0,Fall - 2011,0.0,acharyanagarjunauniversity,0.8900,SUNY Stony Brook
6,nachiappan123,electricalengineering,0.0,45.0,100.0,0.0,157.0,163.0,0.0,3.0,Fall - 2015,0.0,acharyanagarjunauniversity,0.8004,Arizona State University
7,username123456,managementinformationsystem,0.0,0.0,95.0,0.0,143.0,162.0,0.0,3.0,Spring - 2014,0.0,acharyanagarjunauniversity,0.8500,University of Texas Arlington
8,username123456,engineeringmanagement,0.0,0.0,95.0,0.0,143.0,162.0,0.0,3.0,Spring - 2014,0.0,acharyanagarjunauniversity,0.8500,University of North Carolina Charlotte
9,sam_the_killer,electricalandcomputerengineeringelectricalengi...,0.0,0.0,89.0,0.0,370.0,690.0,0.0,4.0,Fall - 2007,0.0,acharyanagarjunauniversity,0.6941,University of Texas Arlington


In [190]:
from sklearn import preprocessing
number = preprocessing.LabelEncoder()
data['userName'] = number.fit_transform(data.userName)
data['major'] = number.fit_transform(data.major)
data['termAndYear'] = number.fit_transform(data.termAndYear)
data['ugCollege'] = number.fit_transform(data.ugCollege)
data['univName'] = number.fit_transform(data.univName)
#data=data.fillna(-999)


In [191]:
result

,userName,major,researchExp,industryExp,toeflScore,internExp,greV,greQ,journalPubs,greA,termAndYear,confPubs,ugCollege,cgpa,univName
0,ninny.rgs,electricalengineeringandcomputerscience,0.0,0.0,101.0,0.0,430.0,740.0,0.0,4.5,Fall - 2011,0.0,abhinavhitechcollegeofengineering,0.7227,University of Southern California
1,ninny.rgs,electricalengineeringandcomputerscience,0.0,0.0,101.0,0.0,159.0,159.0,0.0,4.5,Fall - 2011,0.0,abviiitm,0.7227,New Jersey Institute of Technology
2,aashayshah,masterofengineeringmanagement,0.0,7.0,88.0,0.0,147.0,164.0,0.0,2.5,Spring - 2016,0.0,aceengineeringcollege,0.6200,University of Texas Arlington
3,cjaganmohan,electronicsandcommunication,0.0,0.0,102.0,0.0,140.0,148.0,0.0,3.0,Fall - 2013,0.0,acharyainstituteoftechnology,0.7400,University of Texas Arlington
4,pvskmz,computerscience,0.0,0.0,115.0,0.0,154.0,163.0,0.0,3.0,Fall - 2013,0.0,acharyanagarjunauniversity,0.7100,Arizona State University
5,anu_661,electricalengineeringandcomputerscience,0.0,0.0,105.0,0.0,154.0,163.0,0.0,3.0,Fall - 2011,0.0,acharyanagarjunauniversity,0.8900,SUNY Stony Brook
6,nachiappan123,electricalengineering,0.0,45.0,100.0,0.0,157.0,163.0,0.0,3.0,Fall - 2015,0.0,acharyanagarjunauniversity,0.8004,Arizona State University
7,username123456,managementinformationsystem,0.0,0.0,95.0,0.0,143.0,162.0,0.0,3.0,Spring - 2014,0.0,acharyanagarjunauniversity,0.8500,University of Texas Arlington
8,username123456,engineeringmanagement,0.0,0.0,95.0,0.0,143.0,162.0,0.0,3.0,Spring - 2014,0.0,acharyanagarjunauniversity,0.8500,University of North Carolina Charlotte
9,sam_the_killer,electricalandcomputerengineeringelectricalengi...,0.0,0.0,89.0,0.0,370.0,690.0,0.0,4.0,Fall - 2007,0.0,acharyanagarjunauniversity,0.6941,University of Texas Arlington


In [192]:
from sklearn.model_selection import train_test_split
Y = data['univName']
X = data.drop('univName',1)

In [193]:
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

In [194]:
rfc = RandomForestClassifier(n_jobs=-1, class_weight="balanced_subsample",max_depth=5)
boruta_selector = BorutaPy(rfc, n_estimators="auto", verbose=2,max_iter=100)
x=X.iloc[:,:].values
#y=Y.iloc[:,0].values
boruta_selector.fit(x,Y)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	7
Tentative: 	1
Rejected: 	6
Iteration: 	9 / 100
Confirmed: 	7
Tentative: 	1
Rejected: 	6
Iteration: 	10 / 100
Confirmed: 	7
Tentative: 	1
Rejected: 	6
Iteration: 	11 / 100
Confirmed: 	7
Tentative: 	1
Rejected: 	6
Iteration: 	12 / 100
Confirmed: 	7
Tentative: 	1
Rejected: 	6
Iteration: 	13 / 100
Confirmed: 	7
Tentative: 	1
Rejected: 	6
Iteration: 	14 / 100
Confirmed: 	7
Tentative: 	1
Rejected: 	6
Iteration: 	15 / 100
Confirmed: 	7
Tentative: 	1
Rejected: 	6
Iteration: 	16 / 100
Confirmed: 	7
Tentative: 	1
Rejected: 	6
Iteration:

BorutaPy(alpha=0.05,
     estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
            criterion='gini', max_depth=5, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=80, n_jobs=-1, oob_score=False,
            random_state=RandomState(MT19937) at 0x21BBF8DEDB0, verbose=0,
            warm_start=False),
     max_iter=100, n_estimators='auto', perc=100,
     random_state=RandomState(MT19937) at 0x21BBF8DEDB0, two_step=True,
     verbose=2)

In [195]:
boruta_selector.n_features_ 

7

In [196]:
boruta_selector.ranking_

array([2, 3, 4, 6, 1, 5, 1, 1, 8, 1, 1, 7, 1, 1])

In [197]:
boruta_selector.support_

array([False, False, False, False,  True, False,  True,  True, False,
        True,  True, False,  True,  True])

In [198]:
X_filtered=X[X.columns[boruta_selector.support_]]


In [199]:
X_filtered

,toeflScore,greV,greQ,greA,termAndYear,ugCollege,cgpa
0,101.0,430.0,740.0,4.5,14,622,0.7227
1,101.0,159.0,159.0,4.5,14,623,0.7227
2,88.0,147.0,164.0,2.5,39,624,0.6200
3,102.0,140.0,148.0,3.0,17,625,0.7400
4,115.0,154.0,163.0,3.0,17,626,0.7100
5,105.0,154.0,163.0,3.0,14,626,0.8900
6,100.0,157.0,163.0,3.0,21,626,0.8004
7,95.0,143.0,162.0,3.0,37,626,0.8500
8,95.0,143.0,162.0,3.0,37,626,0.8500
9,89.0,370.0,690.0,4.0,10,626,0.6941


In [200]:
#After applying boruta feature selection algorithm we observed that the important features are: 
#'toeflScore','greV ','greQ','greA','termAndYear','ugCollege','cgpa'.
#Therefore we can remove the unimportant features.
output=result[['userName','toeflScore','greV','greQ','greA','termAndYear','ugCollege','cgpa','univName']]


In [201]:
output

,userName,toeflScore,greV,greQ,greA,termAndYear,ugCollege,cgpa,univName
0,ninny.rgs,101.0,430.0,740.0,4.5,Fall - 2011,abhinavhitechcollegeofengineering,0.7227,University of Southern California
1,ninny.rgs,101.0,159.0,159.0,4.5,Fall - 2011,abviiitm,0.7227,New Jersey Institute of Technology
2,aashayshah,88.0,147.0,164.0,2.5,Spring - 2016,aceengineeringcollege,0.6200,University of Texas Arlington
3,cjaganmohan,102.0,140.0,148.0,3.0,Fall - 2013,acharyainstituteoftechnology,0.7400,University of Texas Arlington
4,pvskmz,115.0,154.0,163.0,3.0,Fall - 2013,acharyanagarjunauniversity,0.7100,Arizona State University
5,anu_661,105.0,154.0,163.0,3.0,Fall - 2011,acharyanagarjunauniversity,0.8900,SUNY Stony Brook
6,nachiappan123,100.0,157.0,163.0,3.0,Fall - 2015,acharyanagarjunauniversity,0.8004,Arizona State University
7,username123456,95.0,143.0,162.0,3.0,Spring - 2014,acharyanagarjunauniversity,0.8500,University of Texas Arlington
8,username123456,95.0,143.0,162.0,3.0,Spring - 2014,acharyanagarjunauniversity,0.8500,University of North Carolina Charlotte
9,sam_the_killer,89.0,370.0,690.0,4.0,Fall - 2007,acharyanagarjunauniversity,0.6941,University of Texas Arlington


In [202]:
output.to_csv('data.csv')